In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from imblearn.over_sampling import SMOTE
from collections import Counter

In [2]:
df = pd.read_csv("/Python_codes/Pose_estimation_problem/Notebook/Notebooks/csv_data/unbalanced_keypoints.csv")

In [3]:
df = df.copy()
df

,Image_id,Image_label,image_keypoints1,image_keypoints2,image_keypoints3,image_keypoints4,image_keypoints5,image_keypoints6,image_keypoints7,image_keypoints8,...,image_keypoints42,image_keypoints43,image_keypoints44,image_keypoints45,image_keypoints46,image_keypoints47,image_keypoints48,image_keypoints49,image_keypoints50,image_keypoints51
0,Image_1.jpg,sitting,358.75,155.0000,0.997559,392.25,121.125,0.998047,336.75,125.7500,...,0.036072,151.00,564.50,0.029419,354.50,612.50,0.007755,272.50,601.5,0.006439
1,Image_4.jpg,sleeping,368.50,123.3125,0.991211,373.50,114.500,0.919434,355.50,115.9375,...,0.452393,409.25,394.75,0.569824,422.50,476.75,0.157715,404.00,494.5,0.196167
2,Image_6.jpg,sleeping,578.00,346.5000,0.998047,593.00,331.500,0.998047,569.00,341.5000,...,0.835938,493.25,470.50,0.733887,375.75,597.00,0.649902,421.25,548.0,0.587891
3,Image_15.jpg,sleeping,464.50,368.0000,0.996094,503.75,287.500,0.999023,432.00,307.2500,...,0.018127,348.25,773.50,0.015961,530.00,839.50,0.008514,561.50,849.5,0.008064
4,Image_33.jpg,sitting,489.25,223.8750,0.999512,509.00,198.625,0.993652,462.50,197.7500,...,0.793457,708.00,515.00,0.804688,925.00,626.50,0.545898,925.00,634.0,0.561523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1264,Image_12527.jpg,running,713.50,191.6250,0.990234,720.00,185.875,0.986328,708.00,186.1250,...,0.950195,709.50,393.25,0.943359,769.00,396.00,0.882812,721.50,457.0,0.873535
1265,Image_12577.jpg,sleeping,496.25,388.0000,0.977539,469.00,306.000,0.975098,409.50,402.0000,...,0.023727,499.50,460.25,0.015129,582.00,445.75,0.020889,484.00,447.5,0.016403
1266,Image_12594.jpg,sleeping,353.00,161.0000,0.996582,364.00,147.750,0.967773,337.25,144.5000,...,0.960938,327.25,399.75,0.961914,472.00,546.00,0.889160,266.00,560.0,0.891602
1267,Image_12596.jpg,sitting,458.50,194.2500,0.996094,473.00,158.750,0.997559,434.25,185.3750,...,0.070312,522.50,548.50,0.058350,676.00,534.00,0.017517,570.00,537.0,0.016159


In [4]:
df.shape

(1269, 53)

Here image_id is a non_numerical data and Image_label is also needs to be a numerical for categorizing and applying smote so labelencoder can be used to transform this non_numerical data.

In [5]:
encoder = LabelEncoder()
en_df = df.loc[:,['Image_id','Image_label']].apply(encoder.fit_transform)
en_df.head(9)
en_df

,Image_id,Image_label
0,0,1
1,587,2
2,804,2
3,312,2
4,508,1
...,...,...
1264,282,0
1265,283,2
1266,285,2
1267,286,1


Seperated all the keypoints 

In [6]:
df_all_keypoints = df.iloc[:,2:]
df_all_keypoints

,image_keypoints1,image_keypoints2,image_keypoints3,image_keypoints4,image_keypoints5,image_keypoints6,image_keypoints7,image_keypoints8,image_keypoints9,image_keypoints10,...,image_keypoints42,image_keypoints43,image_keypoints44,image_keypoints45,image_keypoints46,image_keypoints47,image_keypoints48,image_keypoints49,image_keypoints50,image_keypoints51
0,358.75,155.0000,0.997559,392.25,121.125,0.998047,336.75,125.7500,0.978516,466.00,...,0.036072,151.00,564.50,0.029419,354.50,612.50,0.007755,272.50,601.5,0.006439
1,368.50,123.3125,0.991211,373.50,114.500,0.919434,355.50,115.9375,0.988770,380.50,...,0.452393,409.25,394.75,0.569824,422.50,476.75,0.157715,404.00,494.5,0.196167
2,578.00,346.5000,0.998047,593.00,331.500,0.998047,569.00,341.5000,0.981445,635.50,...,0.835938,493.25,470.50,0.733887,375.75,597.00,0.649902,421.25,548.0,0.587891
3,464.50,368.0000,0.996094,503.75,287.500,0.999023,432.00,307.2500,0.923828,659.00,...,0.018127,348.25,773.50,0.015961,530.00,839.50,0.008514,561.50,849.5,0.008064
4,489.25,223.8750,0.999512,509.00,198.625,0.993652,462.50,197.7500,0.998535,526.00,...,0.793457,708.00,515.00,0.804688,925.00,626.50,0.545898,925.00,634.0,0.561523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1264,713.50,191.6250,0.990234,720.00,185.875,0.986328,708.00,186.1250,0.959473,732.50,...,0.950195,709.50,393.25,0.943359,769.00,396.00,0.882812,721.50,457.0,0.873535
1265,496.25,388.0000,0.977539,469.00,306.000,0.975098,409.50,402.0000,0.982910,454.25,...,0.023727,499.50,460.25,0.015129,582.00,445.75,0.020889,484.00,447.5,0.016403
1266,353.00,161.0000,0.996582,364.00,147.750,0.967773,337.25,144.5000,0.995605,374.50,...,0.960938,327.25,399.75,0.961914,472.00,546.00,0.889160,266.00,560.0,0.891602
1267,458.50,194.2500,0.996094,473.00,158.750,0.997559,434.25,185.3750,0.978516,538.00,...,0.070312,522.50,548.50,0.058350,676.00,534.00,0.017517,570.00,537.0,0.016159


Merging keypoints with Image_id in order to specify the features and seperating labels.

In [7]:
X_1 = en_df["Image_id"]
y = en_df["Image_label"]
X = pd.concat([X_1,df_all_keypoints], axis=1)

In [8]:
X

,Image_id,image_keypoints1,image_keypoints2,image_keypoints3,image_keypoints4,image_keypoints5,image_keypoints6,image_keypoints7,image_keypoints8,image_keypoints9,...,image_keypoints42,image_keypoints43,image_keypoints44,image_keypoints45,image_keypoints46,image_keypoints47,image_keypoints48,image_keypoints49,image_keypoints50,image_keypoints51
0,0,358.75,155.0000,0.997559,392.25,121.125,0.998047,336.75,125.7500,0.978516,...,0.036072,151.00,564.50,0.029419,354.50,612.50,0.007755,272.50,601.5,0.006439
1,587,368.50,123.3125,0.991211,373.50,114.500,0.919434,355.50,115.9375,0.988770,...,0.452393,409.25,394.75,0.569824,422.50,476.75,0.157715,404.00,494.5,0.196167
2,804,578.00,346.5000,0.998047,593.00,331.500,0.998047,569.00,341.5000,0.981445,...,0.835938,493.25,470.50,0.733887,375.75,597.00,0.649902,421.25,548.0,0.587891
3,312,464.50,368.0000,0.996094,503.75,287.500,0.999023,432.00,307.2500,0.923828,...,0.018127,348.25,773.50,0.015961,530.00,839.50,0.008514,561.50,849.5,0.008064
4,508,489.25,223.8750,0.999512,509.00,198.625,0.993652,462.50,197.7500,0.998535,...,0.793457,708.00,515.00,0.804688,925.00,626.50,0.545898,925.00,634.0,0.561523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1264,282,713.50,191.6250,0.990234,720.00,185.875,0.986328,708.00,186.1250,0.959473,...,0.950195,709.50,393.25,0.943359,769.00,396.00,0.882812,721.50,457.0,0.873535
1265,283,496.25,388.0000,0.977539,469.00,306.000,0.975098,409.50,402.0000,0.982910,...,0.023727,499.50,460.25,0.015129,582.00,445.75,0.020889,484.00,447.5,0.016403
1266,285,353.00,161.0000,0.996582,364.00,147.750,0.967773,337.25,144.5000,0.995605,...,0.960938,327.25,399.75,0.961914,472.00,546.00,0.889160,266.00,560.0,0.891602
1267,286,458.50,194.2500,0.996094,473.00,158.750,0.997559,434.25,185.3750,0.978516,...,0.070312,522.50,548.50,0.058350,676.00,534.00,0.017517,570.00,537.0,0.016159


In [9]:
y

0       1
1       2
2       2
3       2
4       1
       ..
1264    0
1265    2
1266    2
1267    1
1268    1
Name: Image_label, Length: 1269, dtype: int32

Applying smote for balancing the dataset

In [10]:
smote = SMOTE() 
X_smote, y_Smote = smote.fit_resample(X.astype('float'),y)

In [11]:
X_smote

,Image_id,image_keypoints1,image_keypoints2,image_keypoints3,image_keypoints4,image_keypoints5,image_keypoints6,image_keypoints7,image_keypoints8,image_keypoints9,...,image_keypoints42,image_keypoints43,image_keypoints44,image_keypoints45,image_keypoints46,image_keypoints47,image_keypoints48,image_keypoints49,image_keypoints50,image_keypoints51
0,0.000000,358.750000,155.000000,0.997559,392.250000,121.125000,0.998047,336.750000,125.750000,0.978516,...,0.036072,151.000000,564.500000,0.029419,354.500000,612.500000,0.007755,272.500000,601.500000,0.006439
1,587.000000,368.500000,123.312500,0.991211,373.500000,114.500000,0.919434,355.500000,115.937500,0.988770,...,0.452393,409.250000,394.750000,0.569824,422.500000,476.750000,0.157715,404.000000,494.500000,0.196167
2,804.000000,578.000000,346.500000,0.998047,593.000000,331.500000,0.998047,569.000000,341.500000,0.981445,...,0.835938,493.250000,470.500000,0.733887,375.750000,597.000000,0.649902,421.250000,548.000000,0.587891
3,312.000000,464.500000,368.000000,0.996094,503.750000,287.500000,0.999023,432.000000,307.250000,0.923828,...,0.018127,348.250000,773.500000,0.015961,530.000000,839.500000,0.008514,561.500000,849.500000,0.008064
4,508.000000,489.250000,223.875000,0.999512,509.000000,198.625000,0.993652,462.500000,197.750000,0.998535,...,0.793457,708.000000,515.000000,0.804688,925.000000,626.500000,0.545898,925.000000,634.000000,0.561523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,120.779214,547.084116,394.273561,0.888745,529.360687,325.813270,0.958001,480.487889,419.655461,0.886445,...,0.018289,716.148708,534.582795,0.023720,698.640634,538.362742,0.012838,717.383588,554.025616,0.016251
2096,1190.999938,524.774081,101.453564,0.774871,524.709555,93.645102,0.215393,517.419249,94.747922,0.758064,...,0.969364,450.177323,311.693578,0.970168,588.322538,412.612874,0.924395,365.709524,334.080534,0.925403
2097,224.358805,752.195608,303.833090,0.508522,745.410587,277.754833,0.507783,747.275891,302.604775,0.404901,...,0.060078,849.176195,549.606688,0.040058,768.521551,567.869622,0.021519,835.827054,563.934277,0.016781
2098,720.519152,353.524734,265.025511,0.341082,346.008974,258.674769,0.205979,343.309964,264.202119,0.256245,...,0.714438,669.679858,343.266395,0.847837,734.765158,291.245404,0.726429,738.794204,309.304273,0.816910


In [12]:
y_Smote

0       1
1       2
2       2
3       2
4       1
       ..
2095    1
2096    1
2097    1
2098    1
2099    1
Name: Image_label, Length: 2100, dtype: int32

The Counter object can be used to summarize the number of examples in each class and here we can compare the number of expamles for unbalanced and balanced labels. 

In [13]:
Counter(y)

Counter({1: 379, 2: 700, 0: 190})

In [14]:
Counter(y_Smote)

Counter({1: 700, 2: 700, 0: 700})

In [15]:
balanced_df = pd.concat([X_smote,y_Smote], axis=1)
balanced_df

,Image_id,image_keypoints1,image_keypoints2,image_keypoints3,image_keypoints4,image_keypoints5,image_keypoints6,image_keypoints7,image_keypoints8,image_keypoints9,...,image_keypoints43,image_keypoints44,image_keypoints45,image_keypoints46,image_keypoints47,image_keypoints48,image_keypoints49,image_keypoints50,image_keypoints51,Image_label
0,0.000000,358.750000,155.000000,0.997559,392.250000,121.125000,0.998047,336.750000,125.750000,0.978516,...,151.000000,564.500000,0.029419,354.500000,612.500000,0.007755,272.500000,601.500000,0.006439,1
1,587.000000,368.500000,123.312500,0.991211,373.500000,114.500000,0.919434,355.500000,115.937500,0.988770,...,409.250000,394.750000,0.569824,422.500000,476.750000,0.157715,404.000000,494.500000,0.196167,2
2,804.000000,578.000000,346.500000,0.998047,593.000000,331.500000,0.998047,569.000000,341.500000,0.981445,...,493.250000,470.500000,0.733887,375.750000,597.000000,0.649902,421.250000,548.000000,0.587891,2
3,312.000000,464.500000,368.000000,0.996094,503.750000,287.500000,0.999023,432.000000,307.250000,0.923828,...,348.250000,773.500000,0.015961,530.000000,839.500000,0.008514,561.500000,849.500000,0.008064,2
4,508.000000,489.250000,223.875000,0.999512,509.000000,198.625000,0.993652,462.500000,197.750000,0.998535,...,708.000000,515.000000,0.804688,925.000000,626.500000,0.545898,925.000000,634.000000,0.561523,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,120.779214,547.084116,394.273561,0.888745,529.360687,325.813270,0.958001,480.487889,419.655461,0.886445,...,716.148708,534.582795,0.023720,698.640634,538.362742,0.012838,717.383588,554.025616,0.016251,1
2096,1190.999938,524.774081,101.453564,0.774871,524.709555,93.645102,0.215393,517.419249,94.747922,0.758064,...,450.177323,311.693578,0.970168,588.322538,412.612874,0.924395,365.709524,334.080534,0.925403,1
2097,224.358805,752.195608,303.833090,0.508522,745.410587,277.754833,0.507783,747.275891,302.604775,0.404901,...,849.176195,549.606688,0.040058,768.521551,567.869622,0.021519,835.827054,563.934277,0.016781,1
2098,720.519152,353.524734,265.025511,0.341082,346.008974,258.674769,0.205979,343.309964,264.202119,0.256245,...,669.679858,343.266395,0.847837,734.765158,291.245404,0.726429,738.794204,309.304273,0.816910,1


Exporting csv for balanced keypoints

In [16]:
balanced_df.to_csv('/Python_codes/Pose_estimation_problem/Notebook/Notebooks/csv_data/Balanced_keypoints.csv', index=False) #